To efficiently run the infereencing on remote servers, we use a Dataset module from python to store tokenized comments, which significantly speeds up the training/inferencing process. Due to the size of the unlabeled dataset, they are split by time and are stored in 6 separate pickle files

In [2]:
import os
import pandas as pd
import numpy as np
import glob
from datetime import date
import datetime
import torch
import time
import sys
import tqdm
from datasets import Dataset
from transformers import AutoTokenizer

In [1]:
files = [
    '20200528_20200713_all_comments.pickle',
    '20200713_20200813_all_comments.pickle',
    '20200201_20200528_all_comments.pickle',
    '20201129_20210201_all_comments.pickle',
    '20201025_20201128_all_comments.pickle',
    '20200813_20201025_all_comments.pickle']

In [3]:
file = files[0]
fname = '/Users/yujia/research/stance/data/comments/{}'.format(file)
df = pd.read_pickle(fname)
df = df.dropna(subset=['body', 'id'])
df = df[(df['author'] != '[deleted]') & (df['author'] != '[removed]') & (df['body'] != '[deleted]') & (df['body'] != '[removed]')]
print(len(df))

12326066


In [24]:
df

,created_utc,parent_id,link_id,id,author,body,subreddit,link,score,author_flair_text,vulgarity,civility,namecalling,stereotype,demeaning,comment_len
0,1590724419,t1_fs5o12b,t3_grqxhn,fs5xqx1,brdwatchr,Your statements were so vague that no one woul...,uspolitics,/r/uspolitics/comments/grqxhn/trump_forgets_we...,1,NaN,0.143654,0.987132,0.950770,0.040720,0.257383,90
1,1590722281,t3_gsjyd6,t3_gsjyd6,fs5udiu,VooDooOperator,Who wrote this headline? A toaster?,uspolitics,/r/uspolitics/comments/gsjyd6/klobuchar_ate_a_...,1,NaN,0.513056,0.963877,0.125301,0.046948,0.065201,6
2,1590720172,t3_gsjyd6,t3_gsjyd6,fs5qv2e,Popular-Way,Trumpie pretending so he can divide Democrats,uspolitics,/r/uspolitics/comments/gsjyd6/klobuchar_ate_a_...,1,NaN,0.078625,0.969522,0.573066,0.078954,0.053198,7
3,1590719247,t1_fs5bha1,t3_gshwx3,fs5pad9,exkallibur,He's failed his entire life and is now President.,uspolitics,/r/uspolitics/comments/gshwx3/trumps_social_me...,1,NaN,0.463623,0.981085,0.938620,0.035715,0.084146,9
4,1590718509,t1_fs2zwjr,t3_grqxhn,fs5o12b,droopus,"Other than specificities, my statement is accu...",uspolitics,/r/uspolitics/comments/grqxhn/trump_forgets_we...,1,NaN,0.186357,0.175046,0.122400,0.045925,0.058365,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3908763,1585713612,t1_fm253jk,t3_fslazd,fm35sci,Magewic,Because the alternatives are even worse. Trump...,politics,/r/politics/comments/fslazd/poll_57_of_voters_...,1,:flag-ny: New York,0.114492,0.983493,0.909322,0.017502,0.034151,42
3908764,1585713610,t3_fsinqs,t3_fsinqs,fm35s8v,Pinkglittersparkles,Fuck the GOP.,politics,/r/politics/comments/fsinqs/marco_rubio_ripped...,1,:flag-il: Illinois,0.608223,0.978564,0.830394,0.056317,0.064667,3
3908765,1585713610,t1_fm35jff,t3_fsrtkx,fm35s8r,DawnSennin,"Fortunately enough, one doesn't have to imagin...",politics,/r/politics/comments/fsrtkx/joe_rogan_says_the...,1,NaN,0.209710,0.988254,0.958252,0.281177,0.042422,40
3908766,1585713609,t1_fm35dt8,t3_fsr5yh,fm35s6w,PHDreseacher,They can't keep people alive without the hospi...,politics,/r/politics/comments/fsr5yh/trump_projects_up_...,1,NaN,0.160026,0.839452,0.791004,0.045447,0.061701,9


In [4]:
df1 = df.iloc[6000000:, :]
# df2 = df.iloc[3000000:, :]

dataset = Dataset.from_pandas(df1)
# dataset = Dataset.load_from_disk('/Users/yujia/research/civility/datasets/20200201_20200528')

dataset = dataset.remove_columns([ 'author', 'author_flair_text', '__index_level_0__', 'created_utc', 'link', 'link_id', 'parent_id', 'score', 'subreddit', 'vulgarity', 'civility', 'namecalling', 'stereotype', 'demeaning'])
# dataset = dataset.remove_columns([ 'num_comments', 'removed_by_category', 'selftext', 'title', 'upvote_ratio', 'url'])

dataset

Dataset({
    features: ['id', 'body', 'comment_len'],
    num_rows: 6326066
})

In [5]:
datasets_dir = "/Users/yujia/research/civility/datasets"
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

dataset = dataset.map(lambda e: tokenizer(e['body'], truncation=True, padding='max_length'), batched=True)
dataset.save_to_disk(os.path.join(datasets_dir, file+"_second"))
dataset

Parameter 'function'=<function <lambda> at 0x7fb93faa00e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/6327 [00:00<?, ?ba/s]

Dataset({
    features: ['attention_mask', 'body', 'comment_len', 'id', 'input_ids', 'token_type_ids'],
    num_rows: 6326066
})

In [6]:
dataset

406377990

406377990